In [1]:
!pip install torch torchvision tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

In [ ]:
import os
import zipfile
import shutil
from sklearn.model_selection import train_test_split

# ====================================
# 1. Unzip the uploaded file
# ====================================
zip_path = "/content/garbage_calssification_dataset.zip"   # Change if needed
extract_path = "/content/extracted"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("📦 ZIP extracted!")

# ====================================
# 2. Find the inner dataset folder
# (because it's nested twice)
# ====================================
base = os.path.join(extract_path, "Garbage classification", "Garbage classification")

if not os.path.exists(base):
    raise Exception("❌ Dataset structure not found. Check ZIP contents.")

print("📁 Dataset found at:", base)

# ====================================
# 3. Create new clean dataset folders
# ====================================
output_dir = "/content/dataset"
train_dir  = os.path.join(output_dir, "train")
test_dir   = os.path.join(output_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

classes = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]

for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

print("📂 Train/Test folders created!")

# ====================================
# 4. Split each class 80% train / 20% test
# ====================================
for cls in classes:
    class_path = os.path.join(base, cls)
    images = os.listdir(class_path)

    images = [img for img in images if img.lower().endswith((".jpg", ".jpeg", ".png"))]

    train_imgs, test_imgs = train_test_split(images, test_size=0.2, random_state=42)

    # Copy train images
    for img in train_imgs:
        shutil.copy(os.path.join(class_path, img),
                    os.path.join(train_dir, cls, img))

    # Copy test images
    for img in test_imgs:
        shutil.copy(os.path.join(class_path, img),
                    os.path.join(test_dir, cls, img))

    print(f"✔ {cls}: {len(train_imgs)} train / {len(test_imgs)} test")

print("\n🎉 Dataset restructuring complete!")
print("Your dataset is ready at: /content/dataset")


In [ ]:
data_dir = "/content/dataset"

In [ ]:
data_dir = "/content/dataset"

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transforms)
val_dataset   = datasets.ImageFolder(os.path.join(data_dir, "test"),  transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32)


In [ ]:
num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes)


In [ ]:
model = models.mobilenet_v2(weights="IMAGENET1K_V2")
model.classifier[1] = nn.Linear(model.last_channel, num_classes)

model = model.to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
num_epochs = 15
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
best_acc = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    loop = tqdm(train_loader, leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        loop.set_description(f"Epoch {epoch+1}/{num_epochs}")
        loop.set_postfix(loss=loss.item(), acc=100.*correct/total)

    # ---- VALIDATION ----
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_acc = 100 * val_correct / val_total
    print(f"Epoch {epoch+1} — Val Accuracy: {val_acc:.2f}%")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "model_best.pth")
        print("✨ Saved new best model!")
